# Data Pipeline - Cell culture media

## Required Python Libraries

In [4]:
import glob
import numpy as np
import pandas as pd
import xlrd

## Input Files

Put media formulation information in the 'input' folder

## Code

In [2]:
# iterate over formulation files
for fn in glob.glob('../../../qFlux-files/media/input_data/*.xlsx'):
    
    # initialize dictionary of values
    media = {}
    
    # load formulation file
    wb = xlrd.open_workbook(fn)
    
    # iterate over sheets
    for sheetname in wb.sheet_names():
        if str(sheetname) != 'REFERENCE':
            
            # load sheet
            sheet = wb.sheet_by_name(str(sheetname))
            
            # metabolite id: column 6, start at row 3
            # exchange reaction names: column 7, start at row 3
            # concentration values: column 8, start at row 3
            for i in range(2,len(sheet.col(6))):
                
                # if reaction already exists, add to current value
                if str(sheet.col(6)[i].value) in media:
                    media[str(sheet.col(6)[i].value)]['value'] += float(sheet.col(7)[i].value)

                # if reaction doesn't exist, add new entry
                else:
                    media[str(sheet.col(6)[i].value)] = {}
                    media[str(sheet.col(6)[i].value)]['met'] = str(sheet.col(5)[i].value)
                    media[str(sheet.col(6)[i].value)]['value'] = float(sheet.col(7)[i].value)
    
    # create media file
    with open('%s.csv' % fn.split('/')[-1].split('.xlsx')[0], 'w') as f:
        
        # header
        f.write('METABOLITE,REACTION,CONCENTRATION [M]\n')
        
        # add entries
        for reaction in sorted(media.keys()):
            f.write('%s,%s,%g\n' % (media[reaction]['met'],reaction,media[reaction]['value']))